<a href="https://colab.research.google.com/github/Vishu52/9d9b93d3-2a90-4e7f-96c1-7607df7a178f/blob/main/Flask_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Hello World Flask App
from flask import Flask, render_template, request, session, redirect, send_from_directory, jsonify
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user
from flask_socketio import SocketIO, send, emit
import sqlite3, os, random

app = Flask(__name__)
app.secret_key = "secret123"
socketio = SocketIO(app)

# --------------------
# 1. Hello World
# --------------------
@app.route("/hello")
def hello():
    return "Hello, World!"

# --------------------
# 2. Static Pages
# --------------------
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/about")
def about():
    return render_template("about.html")

# --------------------
# 3. URL Parameters
# --------------------
@app.route("/hello/<name>")
def greet(name):
    return f"Hello, {name}!"

# --------------------
# 4. Form Input
# --------------------
@app.route("/form", methods=["GET", "POST"])
def form():
    name = ""
    if request.method == "POST":
        name = request.form["username"]
    return render_template("form.html", name=name)

# --------------------
# 5. Sessions
# --------------------
@app.route("/login", methods=["GET", "POST"])
def login_page():
    if request.method == "POST":
        session["user"] = request.form["name"]
        return redirect("/dashboard")
    return "<form method='POST'><input name='name'><button>Login</button></form>"

@app.route("/dashboard")
def dashboard():
    return f"Welcome {session.get('user', 'Guest')}"

# --------------------
# 6. File Upload
# --------------------
app.config["UPLOAD_FOLDER"] = "uploads/"

@app.route("/upload", methods=["GET", "POST"])
def upload():
    filename = ""
    if request.method == "POST":
        f = request.files["file"]
        filename = f.filename
        f.save(os.path.join(app.config["UPLOAD_FOLDER"], filename))
    return render_template("upload.html", filename=filename)

@app.route("/uploads/<filename>")
def uploads(filename):
    return send_from_directory("uploads", filename)

# --------------------
# 7. SQLite CRUD
# --------------------
def db():
    conn = sqlite3.connect("items.db")
    conn.row_factory = sqlite3.Row
    return conn

@app.route("/items")
def items():
    data = db().execute("SELECT * FROM items").fetchall()
    return render_template("items.html", items=data)

@app.route("/items/add", methods=["POST"])
def items_add():
    db().execute("INSERT INTO items(name) VALUES (?)", (request.form["name"],))
    db().commit()
    return redirect("/items")

@app.route("/items/delete/<id>")
def items_delete(id):
    db().execute("DELETE FROM items WHERE id=?", (id,))
    db().commit()
    return redirect("/items")

# --------------------
# 8. Authentication (Flask-Login)
# --------------------
login_manager = LoginManager(app)

class User(UserMixin):
    pass

USERS = {"admin": {"password": "1234"}}

@login_manager.user_loader
def load(uid):
    u = User()
    u.id = uid
    return u

@app.route("/auth", methods=["GET", "POST"])
def auth():
    if request.method == "POST":
        u = request.form["username"]
        if u in USERS and USERS[u]["password"] == request.form["password"]:
            user = User()
            user.id = u
            login_user(user)
            return redirect("/auth/dashboard")
    return render_template("login.html")

@app.route("/auth/dashboard")
@login_required
def auth_dash():
    return "Logged in!"

@app.route("/logout")
def logout():
    logout_user()
    return redirect("/auth")

# --------------------
# 9. RESTful API CRUD
# --------------------
api_data = []

@app.route("/api", methods=["GET"])
def api_get():
    return jsonify(api_data)

@app.route("/api", methods=["POST"])
def api_add():
    api_data.append(request.json)
    return jsonify({"status": "added"})

@app.route("/api/<int:i>", methods=["PUT"])
def api_update(i):
    api_data[i] = request.json
    return jsonify({"status": "updated"})

@app.route("/api/<int:i>", methods=["DELETE"])
def api_delete(i):
    api_data.pop(i)
    return jsonify({"status": "deleted"})

# --------------------
# 10. Error Handling
# --------------------
@app.errorhandler(404)
def notfound(e):
    return "404 - Page Not Found", 404

@app.errorhandler(500)
def servererror(e):
    return "500 - Server Error", 500

# --------------------
# 11. Real-time Chat (SocketIO)
# --------------------
@app.route("/chat")
def chat():
    return render_template("chat.html")

@socketio.on("message")
def chat_message(msg):
    send(msg, broadcast=True)

# --------------------
# 12. Real-time Data Updates
# --------------------
@app.route("/live")
def live():
    return render_template("live.html")

@socketio.on("connect")
def send_random():
    while True:
        socketio.sleep(2)
        emit("number", random.randint(1, 100))

# --------------------
# 13. Real-time Notifications
# --------------------
@app.route("/notify")
def notify():
    socketio.emit("note", "New Update!")
    return "Sent!"

@app.route("/notify-view")
def notify_view():
    return render_template("notify.html")

# --------------------
# Run App
# --------------------
if __name__ == "__main__":
    socketio.run(app, debug=True)
